In [68]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

# one hot encoder
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

from sklearn import preprocessing
labelencoder = preprocessing.LabelEncoder()

from pygam import LinearGAM, s, f, te

# ignore warning
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.filterwarnings('ignore')

# MD

In [84]:
df = pd.read_csv('./sentiment_output/sentiment_MD22.csv')
#df_new = df[df.columns[:6]]
df_new = df.copy()
df_new['date'] = df_new['date'].astype(str)
df_new.head(5)

,company_name,Group,ticker,file_name,date,WA_sentiment,D0,D-1,D+1,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
0,Admiral Group,Motor/Personal,ADM.L,20110302_Admiral_Group_PLC-_Earnings_Call_2011...,2011-03-02,0.000395,1655.0,-0.031030,0.013897,-0.020130,0.015106,-0.003612,-0.000604,-0.011350,-0.003021,-0.029326,-0.060423,-0.030463,-0.045317
1,Admiral Group,Motor/Personal,ADM.L,20110824_Admiral_Group_PLC-_Earnings_Call_2011...,2011-08-24,0.001340,1353.0,-0.118567,-0.054693,-0.110454,-0.025868,-0.094983,-0.005913,-0.125969,0.011086,-0.031496,0.008130,-0.117417,-0.028825
2,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,-0.000072,887.5,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
3,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,0.000333,887.5,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
4,Admiral Group,Motor/Personal,ADM.L,20120307_Admiral_Group_PLC-_Earnings_Call_2012...,2012-03-07,0.002525,1144.0,0.100000,0.015734,0.095785,0.012238,0.080264,0.022727,0.062210,0.036713,0.103182,0.020979,0.179990,0.014860


In [13]:
def same_direction(df, day_type):
    df_same = df[((df['WA_sentiment']<0) & (df[day_type]<0)) | ((df['WA_sentiment']>0) & (df[day_type]>0))]
    return df_same

In [75]:
for day in day_list:
    print(f"------------- {day}")
    df_run = same_direction(df_new, day)
    df_run['group'] = labelencoder.fit_transform(df_run['Group'])

    X = df_run[['WA_sentiment','group']].values.tolist()
    Y = df_run[day]

    gam = LinearGAM(s(0)+f(1)).fit(X, Y)
    print(gam.summary())
    print("                      ")

------------- D-1
LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                     18.9709
Link Function:                     IdentityLink Log Likelihood:                               -799603.5571
Number of Samples:                          966 AIC:                                           1599247.056
                                                AICc:                                         1599247.9423
                                                GCV:                                                0.0006
                                                Scale:                                              0.0006
                                                Pseudo R-Squared:                                   0.1718
Feature Function   

In [90]:
for day in day_list:
    print(f"------------- {day}")
    df_run = same_direction(df_new, day)
    df_run['group'] = labelencoder.fit_transform(df_run['Group'])

    X = df_run[['WA_sentiment','group']].values.tolist()
    Y = df_run[day]

    gam = LinearGAM(s(0)+f(1)).fit(X, Y)
    print(gam.summary())
    print("                      ")

------------- D-1
LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                     18.9709
Link Function:                     IdentityLink Log Likelihood:                               -799603.5571
Number of Samples:                          966 AIC:                                           1599247.056
                                                AICc:                                         1599247.9423
                                                GCV:                                                0.0006
                                                Scale:                                              0.0006
                                                Pseudo R-Squared:                                   0.1718
Feature Function   

In [76]:
X_col = ['WA_sentiment']
df_dummies = pd.get_dummies(df_new,prefix=['Group'], columns = ['Group'], drop_first=False)
dummies_category = df_dummies.columns[-10:]
X_col.extend(dummies_category)
print(X_col)

day_list = df_new.columns[7:].tolist()
print(day_list)

['WA_sentiment', 'Group_European (Re)Insurers', 'Group_Global Commercial', 'Group_Japanese & Pacific', 'Group_LN Equity', 'Group_Life Groups & Retail Life', 'Group_London Market', 'Group_Motor/Personal', 'Group_Other insurers', 'Group_Run-off', 'Group_US - Specialty/P&C/Reinsurance']
['D-1', 'D+1', 'D-2', 'D+2', 'D-3', 'D+3', 'D-5', 'D+5', 'D-10', 'D+10', 'D-15', 'D+15']


In [89]:
formula = 's(WA_sentiment) '
for cat in dummies_category:
    new_var = f'+ s({cat}) + te(WA_sentiment,{cat})'
    formula += new_var
print(formula)

for day in day_list:
    print(f"------------- {day}")
    df_run = same_direction(df_dummies, day)
    # df_run['group'] = labelencoder.fit_transform(df_run['Group'])

    X = df_run[X_col].values.tolist()
    Y = df_run[day]

    gam = LinearGAM(s(0) + s(1) + te(0,1)+ s(2) + te(0,2)+ s(3) + te(0,3)+ s(4) + te(0,4)
                    + s(5) + te(0,5)+ s(6) + te(0,6)+ s(7) + te(0,7)+ s(8) + te(0,8)
                    + s(9) + te(0,9)+ s(10) + te(0,10)).fit(X, Y)
    print(gam.summary())
    print("                      ")

s(WA_sentiment) + s(Group_European (Re)Insurers) + te(WA_sentiment,Group_European (Re)Insurers)+ s(Group_Global Commercial) + te(WA_sentiment,Group_Global Commercial)+ s(Group_Japanese & Pacific) + te(WA_sentiment,Group_Japanese & Pacific)+ s(Group_LN Equity) + te(WA_sentiment,Group_LN Equity)+ s(Group_Life Groups & Retail Life) + te(WA_sentiment,Group_Life Groups & Retail Life)+ s(Group_London Market) + te(WA_sentiment,Group_London Market)+ s(Group_Motor/Personal) + te(WA_sentiment,Group_Motor/Personal)+ s(Group_Other insurers) + te(WA_sentiment,Group_Other insurers)+ s(Group_Run-off) + te(WA_sentiment,Group_Run-off)+ s(Group_US - Specialty/P&C/Reinsurance) + te(WA_sentiment,Group_US - Specialty/P&C/Reinsurance)
------------- D-1
LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDi

In [82]:
# formula = 's(0) '
# for i in range(len(dummies_category)):
#     new_var = f'+ s({i+1}) + te(0,{i+1})'
#     formula += new_var
# formula

's(0) + s(1) + te(0,1)+ s(2) + te(0,2)+ s(3) + te(0,3)+ s(4) + te(0,4)+ s(5) + te(0,5)+ s(6) + te(0,6)+ s(7) + te(0,7)+ s(8) + te(0,8)+ s(9) + te(0,9)+ s(10) + te(0,10)'

# QA

In [91]:
df = pd.read_csv('./sentiment_output/sentiment_QA8.csv')
#df_new = df[df.columns[:6]]
df_new = df.copy()
df_new['date'] = df_new['date'].astype(str)
df_new.head(5)

,company_name,Group,ticker,file_name,date,WA_sentiment,Close,D-1,D+1,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
0,Admiral Group,Motor/Personal,ADM.L,20110302_Admiral_Group_PLC-_Earnings_Call_2011...,2011-03-02,0.014586,1655.0,-0.031030,0.013897,-0.020130,0.015106,-0.003612,-0.000604,-0.011350,-0.003021,-0.029326,-0.060423,-0.030463,-0.045317
1,Admiral Group,Motor/Personal,ADM.L,20110824_Admiral_Group_PLC-_Earnings_Call_2011...,2011-08-24,0.021825,1353.0,-0.118567,-0.054693,-0.110454,-0.025868,-0.094983,-0.005913,-0.125969,0.011086,-0.031496,0.008130,-0.117417,-0.028825
2,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,0.013218,887.5,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
3,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,0.025269,887.5,-0.256077,-0.076056,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
4,Admiral Group,Motor/Personal,ADM.L,20120307_Admiral_Group_PLC-_Earnings_Call_2012...,2012-03-07,0.037031,1144.0,0.100000,0.015734,0.095785,0.012238,0.080264,0.022727,0.062210,0.036713,0.103182,0.020979,0.179990,0.014860


In [92]:
for day in day_list:
    print(f"------------- {day}")
    df_run = same_direction(df_new, day)
    df_run['group'] = labelencoder.fit_transform(df_run['Group'])

    X = df_run[['WA_sentiment','group']].values.tolist()
    Y = df_run[day]

    gam = LinearGAM(s(0)+f(1)).fit(X, Y)
    print(gam.summary())
    print("                      ")

------------- D-1
LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                     18.6071
Link Function:                     IdentityLink Log Likelihood:                               -909620.3221
Number of Samples:                          975 AIC:                                          1819279.8584
                                                AICc:                                         1819280.7051
                                                GCV:                                                0.0005
                                                Scale:                                              0.0005
                                                Pseudo R-Squared:                                    0.212
Feature Function   

In [93]:
X_col = ['WA_sentiment']
df_dummies = pd.get_dummies(df_new,prefix=['Group'], columns = ['Group'], drop_first=False)
dummies_category = df_dummies.columns[-10:]
X_col.extend(dummies_category)
print(X_col)

day_list = df_new.columns[7:].tolist()
print(day_list)

['WA_sentiment', 'Group_European (Re)Insurers', 'Group_Global Commercial', 'Group_Japanese & Pacific', 'Group_LN Equity', 'Group_Life Groups & Retail Life', 'Group_London Market', 'Group_Motor/Personal', 'Group_Other insurers', 'Group_Run-off', 'Group_US - Specialty/P&C/Reinsurance']
['D-1', 'D+1', 'D-2', 'D+2', 'D-3', 'D+3', 'D-5', 'D+5', 'D-10', 'D+10', 'D-15', 'D+15']


In [95]:
formula = 's(WA_sentiment) '
for cat in dummies_category:
    new_var = f'+ s({cat}) + te(WA_sentiment,{cat})'
    formula += new_var
print(formula)

for day in day_list:
    print(f"------------- {day}")
    df_run = same_direction(df_dummies, day)
    # df_run['group'] = labelencoder.fit_transform(df_run['Group'])

    X = df_run[X_col].values.tolist()
    Y = df_run[day]

    gam = LinearGAM(s(0) + s(1) + te(0,1)+ s(2) + te(0,2)+ s(3) + te(0,3)+ s(4) + te(0,4)
                    + s(5) + te(0,5)+ s(6) + te(0,6)+ s(7) + te(0,7)+ s(8) + te(0,8)
                    + s(9) + te(0,9)+ s(10) + te(0,10)).fit(X, Y)
    print(gam.summary())
    print("                      ")

s(WA_sentiment) + s(Group_European (Re)Insurers) + te(WA_sentiment,Group_European (Re)Insurers)+ s(Group_Global Commercial) + te(WA_sentiment,Group_Global Commercial)+ s(Group_Japanese & Pacific) + te(WA_sentiment,Group_Japanese & Pacific)+ s(Group_LN Equity) + te(WA_sentiment,Group_LN Equity)+ s(Group_Life Groups & Retail Life) + te(WA_sentiment,Group_Life Groups & Retail Life)+ s(Group_London Market) + te(WA_sentiment,Group_London Market)+ s(Group_Motor/Personal) + te(WA_sentiment,Group_Motor/Personal)+ s(Group_Other insurers) + te(WA_sentiment,Group_Other insurers)+ s(Group_Run-off) + te(WA_sentiment,Group_Run-off)+ s(Group_US - Specialty/P&C/Reinsurance) + te(WA_sentiment,Group_US - Specialty/P&C/Reinsurance)
------------- D-1
LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDi